# 영화 기반 영화추천(라이브러리 사용)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import helper

In [2]:
# Import the Movies dataset
movies = pd.read_csv('movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy


In [3]:
#To Count the number of movies in each genres
def count_word(df, ref_col, liste):
    keyword_count = dict()
    for s in liste: keyword_count[s] = 0
    for liste_keywords in df[ref_col].str.split('|'):
        if type(liste_keywords) == float and pd.isnull(liste_keywords): continue
        for s in liste_keywords: 
            if pd.notnull(s): keyword_count[s] += 1
    # convert the dictionary in a list to sort the keywords  by frequency
    keyword_occurences = []
    for k,v in keyword_count.items():
        keyword_occurences.append([k,v])
    keyword_occurences.sort(key = lambda x:x[1], reverse = True)
    return keyword_occurences, keyword_count

In [4]:
genre_labels = set()
for s in movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

In [5]:
#counting how many times each of genres occur:
keyword_occurences, dum = count_word(movies, 'genres', genre_labels)
keyword_occurences

[['Drama', 4365],
 ['Comedy', 3315],
 ['Thriller', 1729],
 ['Action', 1545],
 ['Romance', 1545],
 ['Adventure', 1117],
 ['Crime', 1100],
 ['Horror', 877],
 ['Sci-Fi', 792],
 ['Fantasy', 654],
 ['Children', 583],
 ['Mystery', 543],
 ['Documentary', 495],
 ['Animation', 447],
 ['Musical', 394],
 ['War', 367],
 ['Western', 168],
 ['IMAX', 153],
 ['Film-Noir', 133],
 ['(no genres listed)', 18]]

In [6]:
# Convert Genres in row to Columns
# New Column is created for every genre

movies_genre = movies

for (key,cnt) in keyword_occurences:
    movies_genre.loc[movies_genre['genres'].str.contains(key), key] = 1
    movies_genre[key] = movies_genre[key].fillna(0)


movies_genre.head()

c:\program files\python36\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,movieId,title,genres,Drama,Comedy,Thriller,Action,Romance,Adventure,Crime,...,Children,Mystery,Documentary,Animation,Musical,War,Western,IMAX,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
movies_genre = movies_genre.drop(columns=['genres'])

In [8]:
movies_genre

,movieId,title,Drama,Comedy,Thriller,Action,Romance,Adventure,Crime,Horror,...,Children,Mystery,Documentary,Animation,Musical,War,Western,IMAX,Film-Noir,(no genres listed)
0,1,Toy Story (1995),0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,162672,Mohenjo Daro (2016),1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9121,163056,Shin Godzilla (2016),0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9123,164977,The Gay Desperado (1936),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def get_movie(movies):
    movies_genre_indexing = pd.DataFrame()
    genre_ratings = pd.concat([movies_genre_indexing, movies['movieId'], movies['title']], axis=1)
    return movies_genre_indexing

In [10]:
movies_genre_1k = pd.DataFrame(movies_genre, columns = list(genre_labels))

In [11]:
movies_genre_1k = movies_genre_1k[:1000]

In [12]:
movies_genre_1k

,Film-Noir,Drama,Mystery,Thriller,Documentary,IMAX,Musical,Horror,Comedy,War,Animation,Crime,Western,Fantasy,Sci-Fi,Children,Action,(no genres listed),Adventure,Romance
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0



# 클러스터링 !!!!!!!!!!!!!


In [13]:
from sklearn.cluster import KMeans

In [14]:
predictions_1 = KMeans(n_clusters=10, algorithm='full').fit_predict(movies_genre_1k)

In [15]:
predictions_1

array([5, 5, 1, 4, 1, 7, 1, 5, 9, 3, 4, 1, 5, 2, 9, 0, 4, 1, 1, 0, 7, 0,
       7, 2, 4, 2, 2, 4, 2, 0, 2, 7, 2, 4, 0, 6, 1, 1, 2, 2, 0, 2, 9, 8,
       4, 7, 5, 4, 7, 4, 2, 1, 2, 2, 4, 2, 5, 2, 2, 1, 1, 1, 3, 1, 1, 9,
       9, 8, 2, 4, 3, 6, 0, 2, 2, 4, 8, 4, 8, 4, 9, 1, 1, 9, 2, 1, 4, 3,
       8, 0, 9, 6, 2, 1, 1, 7, 1, 4, 5, 6, 2, 0, 9, 2, 2, 6, 0, 1, 1, 2,
       1, 4, 2, 1, 5, 1, 2, 4, 7, 1, 6, 4, 1, 1, 0, 5, 2, 2, 0, 2, 4, 2,
       9, 2, 2, 8, 1, 5, 0, 3, 2, 6, 9, 7, 7, 8, 2, 4, 5, 3, 8, 3, 9, 1,
       2, 1, 6, 8, 4, 1, 9, 7, 2, 7, 1, 1, 6, 1, 7, 4, 2, 8, 4, 6, 0, 4,
       7, 2, 4, 1, 9, 8, 6, 4, 3, 2, 2, 2, 2, 4, 1, 2, 8, 2, 6, 4, 1, 4,
       2, 9, 1, 2, 2, 1, 4, 2, 1, 8, 9, 1, 5, 1, 7, 2, 2, 1, 5, 0, 6, 0,
       1, 4, 1, 9, 1, 2, 2, 1, 1, 7, 5, 0, 3, 2, 2, 2, 2, 4, 4, 1, 0, 2,
       4, 2, 8, 2, 1, 1, 1, 2, 1, 2, 2, 4, 2, 0, 2, 1, 7, 1, 0, 2, 3, 0,
       1, 1, 0, 2, 2, 4, 4, 9, 8, 1, 2, 2, 8, 2, 1, 5, 2, 2, 3, 8, 0, 8,
       7, 2, 8, 8, 1, 2, 9, 7, 2, 9, 2, 6, 1, 2, 7,

In [24]:
print(len(predictions_1))

1000


In [41]:
movies_genre

,movieId,title,Drama,Comedy,Thriller,Action,Romance,Adventure,Crime,Horror,...,Children,Mystery,Documentary,Animation,Musical,War,Western,IMAX,Film-Noir,(no genres listed)
0,1,Toy Story (1995),0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,162672,Mohenjo Daro (2016),1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9121,163056,Shin Godzilla (2016),0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9123,164977,The Gay Desperado (1936),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
movies_genre['movieId']

0            1
1            2
2            3
3            4
4            5
         ...  
9120    162672
9121    163056
9122    163949
9123    164977
9124    164979
Name: movieId, Length: 9125, dtype: int64

In [43]:
clustered_movie = {}
for i in range(len(predictions_1)):
    clustered_movie[movies_genre['movieId'][i]] = predictions_1[i]

print(clustered_movie)
print(len(clustered_movie))

{1: 5, 2: 5, 3: 1, 4: 4, 5: 1, 6: 7, 7: 1, 8: 5, 9: 9, 10: 3, 11: 4, 12: 1, 13: 5, 14: 2, 15: 9, 16: 0, 17: 4, 18: 1, 19: 1, 20: 0, 21: 7, 22: 0, 23: 7, 24: 2, 25: 4, 26: 2, 27: 2, 28: 4, 29: 2, 30: 0, 31: 2, 32: 7, 34: 2, 35: 4, 36: 0, 37: 6, 38: 1, 39: 1, 40: 2, 41: 2, 42: 0, 43: 2, 44: 9, 45: 8, 46: 4, 47: 7, 48: 5, 49: 4, 50: 7, 52: 4, 53: 2, 54: 1, 55: 2, 57: 2, 58: 4, 59: 2, 60: 5, 61: 2, 62: 2, 63: 1, 64: 1, 65: 1, 66: 3, 68: 1, 69: 1, 70: 9, 71: 9, 72: 8, 73: 2, 74: 4, 76: 3, 77: 6, 78: 0, 79: 2, 80: 2, 81: 4, 82: 8, 83: 4, 84: 8, 85: 4, 86: 9, 87: 1, 88: 1, 89: 9, 92: 2, 93: 1, 94: 4, 95: 3, 96: 8, 97: 0, 98: 9, 99: 6, 100: 2, 101: 1, 102: 1, 103: 7, 104: 1, 105: 4, 107: 5, 108: 6, 110: 2, 111: 0, 112: 9, 113: 2, 114: 2, 116: 6, 117: 0, 118: 1, 119: 1, 121: 2, 122: 1, 123: 4, 124: 2, 125: 1, 126: 5, 129: 1, 130: 2, 131: 4, 132: 7, 135: 1, 137: 6, 140: 4, 141: 1, 144: 1, 145: 0, 146: 5, 147: 2, 148: 2, 149: 0, 150: 2, 151: 4, 152: 2, 153: 9, 154: 2, 155: 2, 156: 8, 157: 1, 158:

In [16]:
m_idx = 0
clustered_movie = {}
for idx in range(len(predictions_1)):
    c_movie = []
    for i in range(len(predictions_1)):
        if idx == i:
            continue
        if predictions_1[idx] == predictions_1[i]:
            c_movie.append(i)
    clustered_movie[idx] = c_movie

# print("clustered_user : ", clustered_user.keys())
print("clustered_movie : ", clustered_movie)

clustered_movie :  {0: [1, 7, 12, 46, 56, 98, 114, 125, 137, 148, 210, 216, 230, 279, 308, 326, 328, 354, 371, 404, 431, 489, 494, 500, 514, 521, 526, 527, 528, 536, 543, 552, 567, 576, 600, 602, 622, 628, 647, 663, 671, 712, 740, 796, 813, 819, 821, 826, 827, 828, 829, 832, 833, 834, 835, 836, 837, 861, 866, 916, 979], 1: [0, 7, 12, 46, 56, 98, 114, 125, 137, 148, 210, 216, 230, 279, 308, 326, 328, 354, 371, 404, 431, 489, 494, 500, 514, 521, 526, 527, 528, 536, 543, 552, 567, 576, 600, 602, 622, 628, 647, 663, 671, 712, 740, 796, 813, 819, 821, 826, 827, 828, 829, 832, 833, 834, 835, 836, 837, 861, 866, 916, 979], 2: [4, 6, 11, 17, 18, 36, 37, 51, 59, 60, 61, 63, 64, 81, 82, 85, 93, 94, 96, 107, 108, 110, 113, 115, 119, 122, 123, 136, 153, 155, 159, 164, 165, 167, 179, 190, 196, 200, 203, 206, 209, 211, 215, 220, 222, 224, 227, 228, 239, 246, 247, 248, 250, 257, 259, 264, 265, 273, 278, 290, 298, 304, 307, 309, 313, 320, 322, 332, 338, 342, 361, 363, 364, 365, 367, 369, 379, 381, 382

# 실습

### (1) title 받아옴(params)

In [33]:
title = "Toy Story (1995)"

### (2) title의 idx 찾기

In [34]:
iidx = -1
for i in range(len(movies_genre)):
    if movies_genre['title'][i] == title:
        iidx = i
        break

### (3) 받아온 영화와 비슷한 영화 가져오기 (movie_list)

In [35]:
movie_list = []

In [36]:
for i in clustered_movie[iidx]:
    movie_list.append(movies_genre['mocieId'][i])
#     print(i, `movies_genre['title'][i])
#     print()

In [37]:
movie_list

['Jumanji (1995)',
 'Tom and Huck (1995)',
 'Balto (1995)',
 'Pocahontas (1995)',
 'Indian in the Cupboard, The (1995)',
 'Muppet Treasure Island (1996)',
 'NeverEnding Story III, The (1994)',
 'Amazing Panda Adventure, The (1995)',
 'Casper (1995)',
 'Free Willy 2: The Adventure Home (1995)',
 'Far From Home: The Adventures of Yellow Dog (1995)',
 'Gumby: The Movie (1995)',
 "Kid in King Arthur's Court, A (1995)",
 'Swan Princess, The (1994)',
 'Baby-Sitters Club, The (1995)',
 'Jungle Book, The (1994)',
 'Lion King, The (1994)',
 'Secret Adventures of Tom Thumb, The (1993)',
 'Black Beauty (1994)',
 'Free Willy (1993)',
 'Lassie (1994)',
 'Nightmare Before Christmas, The (1993)',
 'Pagemaster, The (1994)',
 'Andre (1994)',
 'Aladdin (1992)',
 'Snow White and the Seven Dwarfs (1937)',
 'Beauty and the Beast (1991)',
 'Pinocchio (1940)',
 'Homeward Bound II: Lost in San Francisco (1996)',
 'Aristocats, The (1970)',
 'All Dogs Go to Heaven 2 (1996)',
 'James and the Giant Peach (1996)',

## 끝!!!!!!!!!!!!!!!!!!!!!!!